# Discogs Functions

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogs import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

# Discogs Base Class

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
dutils = discogsUtils()

# Collection Section

In [ ]:
%load_ext autoreload
%autoreload
col = collections(disc)

In [ ]:
#col = collections(disc)
decades=['1920', '1930', '1940', '1950', '1960', '1970', '1980', '1990', '2000', '2010']
decades=['1960', '1970', '1980', '1990', '2000', '2010']
fmat="Compilation"
country="US"
style="Soundtrack"
for decade in decades:
    col.downloadCollection(maxPages=30, style=style, country="US", decade=decade) #, fmat=fmat)
#for year in range(1950, 2018):
#    col.downloadCollectionsByYear(maxPages=2, year=str(year), Ncountries=50, Ngenres=100, Nstyles=100)

#https://www.discogs.com/search/?style_exact=Soundtrack&format_exact=Compilation&limit=250&page=1&country_exact=US
#https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&layout=sm&style_exact=Soundtrack&country_exact=US&format_exact=Compilation&decade=2010&page=1
#https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&layout=sm&style_exact=Soundtrack&country_exact=US&format_exact=Compilation&decade=2010&page=1

In [ ]:
ifile = "/Volumes/Music/Discog/collections/US-1960-Soundtrack-9.p"
from webUtils import getWebData, getHTML
from searchUtils import findPatternExt
verydebug=False
fdata  = getFile(ifile)
bsdata = getHTML(fdata)

albumsDir = disc.getAlbumsDir()

albs = albums(disc)


if True:
    collectionsDir  = col.getCollectionsDir()
    collectionFiles = findPatternExt(collectionsDir, pattern=".", ext=".p")
    print("Found {0} downloaded collection files".format(len(collectionFiles)))
    
    artsDowns = {}
    albsDowns = {}
    
    for i,ifile in enumerate(reversed(collectionFiles)):
        print("\n\n\n")
        print("Parsing {0}/{1}: {2}".format(i,len(collectionFiles), ifile))
        
        bsdata = getHTML(ifile)
        artistDB  = {}

        h4s = bsdata.findAll("h4")
        if verydebug:
            print("Found {0} h4 tags".format(len(h4s)))
        
        for ih4,h4 in enumerate(h4s):
            spans = h4.findAll("span")
            
            #### Get Artist
            ref   = None
            if len(spans) == 0:
                ref = h4.find("a")
            else:
                ref = spans[0].find("a")
                
            if ref is None:
                continue
                
            if artsDowns.get(ref) is True:
                continue
                
            try:
                artisthref = ref.attrs.get('href')
                artist     = ref.text.strip()
            except:
                print("Could not get artist/href from {0}".format(ref))
                continue
                
            
            #### Get Album
        
            try:
                ref        = h4.find("a", {"class": "search_result_title"})
                albumhref  = ref.attrs.get('href')
                album      = ref.text.strip()
            except:
                print("Could not get album/href from {0}".format(ref))
                continue

            
            #print("Artist: {0} ---> {1}".format(artist,artisthref))
            #print("Album:  {0} ---> {1}".format(album,albumhref))

            artistID  = arts.discogsUtils.getArtistID(artisthref)
            modValue  = arts.discogsUtils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
            
            
            ##### Download Artist Data
            url      = arts.getArtistRef(artisthref)
            savename = arts.getArtistSavename(artistID)
            if savename is not None:
                if not isFile(savename):
                    try:
                        arts.downloadArtistURL(url, savename)
                    except:
                        artsDowns[url] = True
                        continue
                #print('\t',url)
                #print('\t',savename)
                
        
            if modValue is None:
                artistModDir = setDir(albumsDir, "NAN")
                artistIDDir = setDir(artistModDir, "NAN")
            else:
                artistModDir = setDir(albumsDir, str(modValue))
                artistIDDir = setDir(artistModDir, artistID)


            ##### Download Album Data
            try:
                mediaID = int(getBasename(albumhref))
            except:
                raise ValueError("Not sure about {0}".format(h4))

            #print('\tAlbumDir:',artistIDDir)
            #print('\tMediaID: ',mediaID)

            if not isDir(artistIDDir):
                mkDir(artistIDDir)
            savename = setFile(artistIDDir, "{0}.p".format(mediaID))
            #print('\tSavename:',savename)
            if isFile(savename):
                continue
     
            #print('\t',savename)
            #print("")
            #continue

            baseURL = disc.discogURL
            url     = urllib.parse.urljoin(baseURL, quote(albumhref))
            
            
            if albsDowns.get(url) is True:
                continue
                
            try:
                albs.downloadAlbumURL(url, savename)
            except:
                albsDowns[url] = True
                continue

In [ ]:
#col.createCollectionsData()
#col.downloadCollections()
#col.parseCollections()
#col.mergeCollections(debug=True, force=True)
#col.createCollectionDBs(debug=True)

In [ ]:
disc.moveAlbumFilesToNewModValue(newModValue=100, oldModValue=500)

# Artist Section

In [ ]:
db = disc.getArtistNameToIDsData()

In [ ]:
#arts = artists(disc)
#art.findKnownArtists(debug=True)
#art.findUnknownArtists(minVal=200, debug=True)
#art.downloadUnknownArtists(debug=True)
#arts.parseArtistFiles()
#art.searchDiscogForArtist("$uicideboy$")
#arts.buildMetadata()

# Albums Metadata

In [ ]:
artistNameToID   = disc.getArtistNameToIDData()
artistNameToIDs  = disc.getArtistNameToIDsData()
artistIDToName   = disc.getArtistIDToNameData()        
artistRefToID    = disc.getArtistRefToIDData()
artistIDToRef    = disc.getArtistIDToRefData()
artistRefToName  = disc.getArtistRefToNameData()
artistNameToRef  = disc.getArtistNameToRefData()
artistNameToRefs = disc.getArtistNameToRefsData()

In [ ]:
albumRefToName  = disc.getAlbumRefToNameData()
albumNameToRef  = disc.getAlbumNameToRefData()

artistIDCoreAlbumIDs   = disc.getArtistIDCoreAlbumNames()
artistIDAlbumIDs       = disc.getArtistIDAlbumNames()



In [ ]:
from collections import Counter
core = ["Albums"]
albumCntr = Counter()


print("{0: <4}{1: <50}{2: <15}{3: <15}{4: <12}{5: <12}{6: <12}{7: <12}{8: <12}{9: <12}".format(
    "#","File","Core IDs-IDs","IDs-IDs","Name-IDs","Name-Refs","ID-Names","ID-Refs","Ref-IDs","Ref-Names"))


for i,dbfile in enumerate(dbfiles):
    db = getFile(dbfile)
    for artistID,artistData in db.items():
        artist      = artistData['Artist']
        artistName  = dutils.getArtistName(artist)
        artistRef   = artistData['URL']
        #print("{0: <20}{1: <50}{2}".format(artistID,artist,artistRef))
        
    print("{0: <4}{1: <50}{2: <15}{3: <15}{4: <12}{5: <12}{6: <12}{7: <12}{8: <12}{9: <12}".format(
        i,dbfile,
        len(artistIDCoreAlbumIDs),len(artistIDAlbumIDs),
        len(albumNameToID),len(albumNameToRef),len(albumIDToName),
        len(albumIDToRef),len(albumRefToID),len(albumRefToName)))



In [ ]:
ifile="/Volumes/Music/Discog/artists/0/9900.p"
fdata = getHTML(getFile(ifile, version=2))
an = artistName(fdata)
an.getArtist()
#name = artistName(fdata)
#name


# Albums

In [ ]:
albs = albums(disc)
albs.parseAlbumFiles()

In [ ]:
getFile('/Volumes/Music/Discog/albums-db/NAN-DB.p')

In [ ]:
import random
albs = albums(disc)
nErrs = False
while nErrs is False:
    nErrs = True
    albs.downloadAlbumDataData(int(random.uniform(0,disc.getMaxModVal()))) #, mediaTypes=None)        
    if False:
        nErrs = False
        print("Sleeping for 60 seconds...")
        sleep(60)
#art.findKnownArtists(debug=True)
#art.findUnknownArtists(minVal=200, debug=True)
#art.downloadUnknownArtists(debug=True)
#arts.parseArtistFiles()
#art.searchDiscogForArtist("$uicideboy$")
#arts.buildMetadata()